# 6407 Скосырский Никита. Вариант 3. Лабораторная работа 4

Tensorflow 2.x

1) Подготовка данных

2) Использование Keras Model API

3) Использование Keras Sequential + Functional API

https://www.tensorflow.org/tutorials

Для выполнения лабораторной работы необходимо установить tensorflow версии 2.0 или выше .

Рекомендуется использовать возможности Colab'а по обучению моделей на GPU.



In [2]:
import os
import tensorflow as tf
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
USE_GPU = True
device = '/device:GPU:0' if USE_GPU else '/cpu:0'

In [4]:
if tf.config.list_physical_devices('GPU'):
    print("GPU доступен!")
else:
    print("GPU не обнаружен. Код будет работать на CPU.")

GPU не обнаружен. Код будет работать на CPU.


In [5]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
print('----')
print(tf.test.gpu_device_name())
print('----')

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7844286575711184862
xla_global_id: -1
]
----

----


In [6]:
print_every = 100

# Подготовка данных
Загрузите набор данных из предыдущей лабораторной работы.

In [7]:
def load_cifar10(num_training=49000, num_validation=1000, num_test=10000):
    """
    Fetch the CIFAR-10 dataset from the web and perform preprocessing to prepare
    it for the two-layer neural net classifier. These are the same steps as
    we used for the SVM, but condensed to a single function.
    """
    # Load the raw CIFAR-10 dataset and use appropriate data types and shapes
    cifar10 = tf.keras.datasets.cifar10.load_data()
    (X_train, y_train), (X_test, y_test) = cifar10
    X_train = np.asarray(X_train, dtype=np.float32)
    y_train = np.asarray(y_train, dtype=np.int32).flatten()
    X_test = np.asarray(X_test, dtype=np.float32)
    y_test = np.asarray(y_test, dtype=np.int32).flatten()

    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    # Normalize the data: subtract the mean pixel and divide by std
    mean_pixel = X_train.mean(axis=(0, 1, 2), keepdims=True)
    std_pixel = X_train.std(axis=(0, 1, 2), keepdims=True)
    X_train = (X_train - mean_pixel) / std_pixel
    X_val = (X_val - mean_pixel) / std_pixel
    X_test = (X_test - mean_pixel) / std_pixel

    return X_train, y_train, X_val, y_val, X_test, y_test

# If there are errors with SSL downloading involving self-signed certificates,
# it may be that your Python version was recently installed on the current machine.
# See: https://github.com/tensorflow/tensorflow/issues/10779
# To fix, run the command: /Applications/Python\ 3.7/Install\ Certificates.command
#   ...replacing paths as necessary.

# Invoke the above function to get our data.
NHW = (0, 1, 2)
X_train, y_train, X_val, y_val, X_test, y_test = load_cifar10()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Train data shape:  (49000, 32, 32, 3)
Train labels shape:  (49000,) int32
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


In [8]:
class Dataset(object):
    def __init__(self, X, y, batch_size, shuffle=False):
        """
        Construct a Dataset object to iterate over data X and labels y
        
        Inputs:
        - X: Numpy array of data, of any shape
        - y: Numpy array of labels, of any shape but with y.shape[0] == X.shape[0]
        - batch_size: Integer giving number of elements per minibatch
        - shuffle: (optional) Boolean, whether to shuffle the data on each epoch
        """
        assert X.shape[0] == y.shape[0], 'Got different numbers of data and labels'
        self.X, self.y = X, y
        self.batch_size, self.shuffle = batch_size, shuffle

    def __iter__(self):
        N, B = self.X.shape[0], self.batch_size
        idxs = np.arange(N)
        if self.shuffle:
            np.random.shuffle(idxs)
        return iter((self.X[i:i+B], self.y[i:i+B]) for i in range(0, N, B))


train_dset = Dataset(X_train, y_train, batch_size=64, shuffle=True)
val_dset = Dataset(X_val, y_val, batch_size=64, shuffle=False)
test_dset = Dataset(X_test, y_test, batch_size=64)

In [9]:
# We can iterate through a dataset like this:
for t, (x, y) in enumerate(train_dset):
    print(t, x.shape, y.shape)
    if t > 5: break

0 (64, 32, 32, 3) (64,)
1 (64, 32, 32, 3) (64,)
2 (64, 32, 32, 3) (64,)
3 (64, 32, 32, 3) (64,)
4 (64, 32, 32, 3) (64,)
5 (64, 32, 32, 3) (64,)
6 (64, 32, 32, 3) (64,)


#  Keras Model Subclassing API


Для реализации собственной модели с помощью Keras Model Subclassing API необходимо выполнить следующие шаги:

1) Определить новый класс, который является наследником tf.keras.Model.

2) В методе __init__() определить все необходимые слои из модуля tf.keras.layer

3) Реализовать прямой проход в методе call() на основе слоев, объявленных в __init__()

Ниже приведен пример использования keras API для определения двухслойной полносвязной сети.

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras

In [10]:
class TwoLayerFC(tf.keras.Model):
    def __init__(self, hidden_size, num_classes):
        super(TwoLayerFC, self).__init__()
        initializer = tf.initializers.VarianceScaling(scale=2.0)
        self.fc1 = tf.keras.layers.Dense(hidden_size, activation='relu',
                                         kernel_initializer=initializer)
        self.fc2 = tf.keras.layers.Dense(num_classes, activation='softmax',
                                         kernel_initializer=initializer)
        self.flatten = tf.keras.layers.Flatten()

    def call(self, x, training=False):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x


def test_TwoLayerFC():
    """ A small unit test to exercise the TwoLayerFC model above. """
    input_size, hidden_size, num_classes = 50, 42, 10
    x = tf.zeros((64, input_size))
    model = TwoLayerFC(hidden_size, num_classes)
    with tf.device(device):
        scores = model(x)
        print(scores.shape)

test_TwoLayerFC()

(64, 10)


Реализуйте трехслойную CNN для вашей задачи классификации.

Архитектура сети:
    
1. Сверточный слой (5 x 5 kernels, zero-padding = 'same')
2. Функция активации ReLU
3. Сверточный слой (3 x 3 kernels, zero-padding = 'same')
4. Функция активации ReLU
5. Полносвязный слой
6. Функция активации Softmax

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Conv2D

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dense

In [11]:
class ThreeLayerConvNet(tf.keras.Model):
    def __init__(self, channel_1, channel_2, num_classes):
        super(ThreeLayerConvNet, self).__init__()
        ########################################################################
        # TODO: Implement the __init__ method for a three-layer ConvNet. You   #
        # should instantiate layer objects to be used in the forward pass.     #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        self.conv_layer1 = tf.keras.layers.Conv2D(channel_1, (5, 5), padding='same', activation='relu')
        self.conv_layer2 = tf.keras.layers.Conv2D(channel_2, (3, 3), padding='same', activation='relu')
        self.flatten_layer = tf.keras.layers.Flatten()
        self.fc_layer = tf.keras.layers.Dense(num_classes, activation='softmax')

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                           END OF YOUR CODE                           #
        ########################################################################

    def call(self, x_input, training_flag=False):
        output_scores = None
        ########################################################################
        # TODO: Implement the forward pass for a three-layer ConvNet. You      #
        # should use the layer objects defined in the __init__ method.         #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        output_scores = self.conv_layer1(x_input)
        output_scores = self.conv_layer2(output_scores)
        output_scores = self.flatten_layer(output_scores)
        output_scores = self.fc_layer(output_scores)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                           END OF YOUR CODE                           #
        ########################################################################
        return output_scores

In [12]:
def test_ThreeLayerConvNet():
    channel_1, channel_2, num_classes = 12, 8, 10
    model = ThreeLayerConvNet(channel_1, channel_2, num_classes)
    with tf.device(device):
        x = tf.zeros((64, 3, 32, 32))
        scores = model(x)
        print(scores.shape)

test_ThreeLayerConvNet()

(64, 10)


Пример реализации процесса обучения:

In [13]:
from importlib import reload
import tensorflow as tf

# Перезагрузка модуля k_nearest_neighbor
reload(tf)

# Импорт функции KNearestNeighbor после перезагрузки модуля
import tensorflow as tf

# Проверка версии TensorFlow
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.16.1


In [14]:
def train_part34(model_init_fn, optimizer_init_fn, num_epochs=1, is_training=False):
    """
    Simple training loop for use with models defined using tf.keras. It trains
    a model for one epoch on the CIFAR-10 training set and periodically checks
    accuracy on the CIFAR-10 validation set.

    Inputs:
    - model_init_fn: A function that takes no parameters; when called it
      constructs the model we want to train: model = model_init_fn()
    - optimizer_init_fn: A function which takes no parameters; when called it
      constructs the Optimizer object we will use to optimize the model:
      optimizer = optimizer_init_fn()
    - num_epochs: The number of epochs to train for

    Returns: Nothing, but prints progress during trainingn
    """
    with tf.device(device):


        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()

        model = model_init_fn()
        optimizer = optimizer_init_fn()

        train_loss = tf.keras.metrics.Mean(name='train_loss')
        train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

        val_loss = tf.keras.metrics.Mean(name='val_loss')
        val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='val_accuracy')

        t = 0
        
        for epoch in range(num_epochs):
    
            # Reset the metrics - https://www.tensorflow.org/alpha/guide/migration_guide#new-style_metrics
            # train_loss.reset_states()
            # train_accuracy.reset_states()
            train_loss.reset_state()
            train_accuracy.reset_state()

            for x_np, y_np in train_dset:
                with tf.GradientTape() as tape:

                    # Use the model function to build the forward pass.
                    scores = model(x_np, training=is_training)
                    loss = loss_fn(y_np, scores)

                    gradients = tape.gradient(loss, model.trainable_variables)
                    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

                    # Update the metrics
                    train_loss.update_state(loss)
                    train_accuracy.update_state(y_np, scores)

                    if t % print_every == 0:
                        val_loss.reset_state()
                        val_accuracy.reset_state()
                        for test_x, test_y in val_dset:
                            # During validation at end of epoch, training set to False
                            prediction = model(test_x, training=False)
                            t_loss = loss_fn(test_y, prediction)

                            val_loss.update_state(t_loss)
                            val_accuracy.update_state(test_y, prediction)

                        template = 'Iteration {}, Epoch {}, Loss: {}, Accuracy: {}, Val Loss: {}, Val Accuracy: {}'
                        print (template.format(t, epoch+1,
                                             train_loss.result(),
                                             train_accuracy.result()*100,
                                             val_loss.result(),
                                             val_accuracy.result()*100))
                    t += 1

In [15]:
hidden_size, num_classes = 4000, 10
learning_rate = 1e-2

def model_init_fn():
    return TwoLayerFC(hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.94515323638916, Accuracy: 3.125, Val Loss: 2.704075574874878, Val Accuracy: 13.0
Iteration 100, Epoch 1, Loss: 2.2210593223571777, Accuracy: 28.016706466674805, Val Loss: 1.9005126953125, Val Accuracy: 37.20000076293945
Iteration 200, Epoch 1, Loss: 2.0719752311706543, Accuracy: 31.895212173461914, Val Loss: 1.916794776916504, Val Accuracy: 38.20000076293945
Iteration 300, Epoch 1, Loss: 2.0008108615875244, Accuracy: 33.85589599609375, Val Loss: 1.863828420639038, Val Accuracy: 37.900001525878906
Iteration 400, Epoch 1, Loss: 1.932950496673584, Accuracy: 35.750465393066406, Val Loss: 1.7198725938796997, Val Accuracy: 42.5
Iteration 500, Epoch 1, Loss: 1.8884373903274536, Accuracy: 36.75773620605469, Val Loss: 1.671434998512268, Val Accuracy: 44.400001525878906
Iteration 600, Epoch 1, Loss: 1.8583064079284668, Accuracy: 37.70798873901367, Val Loss: 1.6952496767044067, Val Accuracy: 43.0
Iteration 700, Epoch 1, Loss: 1.8297995328903198, Accuracy: 38.49857330

Обучите трехслойную CNN. В tf.keras.optimizers.SGD укажите Nesterov momentum = 0.9 .

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/optimizers/SGD

Значение accuracy на валидационной выборке после 1 эпохи обучения должно быть > 50% .

In [16]:
learning_rate = 3e-3
channel_1, channel_2, num_classes = 32, 16, 10

def model_init_fn():
    model = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    model = ThreeLayerConvNet(channel_1, channel_2, num_classes)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return model

def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9, nesterov=True)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return optimizer

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.302265167236328, Accuracy: 15.625, Val Loss: 2.341085433959961, Val Accuracy: 8.800000190734863
Iteration 100, Epoch 1, Loss: 1.9169542789459229, Accuracy: 31.605815887451172, Val Loss: 1.6959242820739746, Val Accuracy: 40.20000076293945
Iteration 200, Epoch 1, Loss: 1.7709877490997314, Accuracy: 37.678794860839844, Val Loss: 1.4992386102676392, Val Accuracy: 48.29999923706055
Iteration 300, Epoch 1, Loss: 1.679482340812683, Accuracy: 40.91050720214844, Val Loss: 1.4625269174575806, Val Accuracy: 47.0
Iteration 400, Epoch 1, Loss: 1.6069707870483398, Accuracy: 43.43437957763672, Val Loss: 1.3615479469299316, Val Accuracy: 50.5
Iteration 500, Epoch 1, Loss: 1.5556151866912842, Accuracy: 45.150325775146484, Val Loss: 1.3295198678970337, Val Accuracy: 53.29999923706055
Iteration 600, Epoch 1, Loss: 1.5231233835220337, Accuracy: 46.303035736083984, Val Loss: 1.2979220151901245, Val Accuracy: 54.20000076293945
Iteration 700, Epoch 1, Loss: 1.4920926094055176, A

# Использование Keras Sequential API для реализации последовательных моделей.

Пример для полносвязной сети:

In [17]:
learning_rate = 1e-2

def model_init_fn():
    input_shape = (32, 32, 3)
    hidden_layer_size, num_classes = 4000, 10
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    layers = [
        tf.keras.layers.Flatten(input_shape=input_shape),
        tf.keras.layers.Dense(hidden_layer_size, activation='relu',
                              kernel_initializer=initializer),
        tf.keras.layers.Dense(num_classes, activation='softmax',
                              kernel_initializer=initializer),
    ]
    model = tf.keras.Sequential(layers)
    return model

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Z:\Programs\Python\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Iteration 0, Epoch 1, Loss: 2.9622654914855957, Accuracy: 9.375, Val Loss: 2.8574187755584717, Val Accuracy: 12.700000762939453
Iteration 100, Epoch 1, Loss: 2.212536096572876, Accuracy: 29.099628448486328, Val Loss: 1.8727930784225464, Val Accuracy: 37.70000076293945
Iteration 200, Epoch 1, Loss: 2.063539743423462, Accuracy: 32.62593078613281, Val Loss: 1.803041934967041, Val Accuracy: 41.10000228881836
Iteration 300, Epoch 1, Loss: 1.9884006977081299, Accuracy: 34.50996398925781, Val Loss: 1.8896706104278564, Val Accuracy: 36.39999771118164
Iteration 400, Epoch 1, Loss: 1.9205865859985352, Accuracy: 36.30767059326172, Val Loss: 1.734802007675171, Val Accuracy: 42.599998474121094
Iteration 500, Epoch 1, Loss: 1.8802872896194458, Accuracy: 37.17252731323242, Val Loss: 1.6493953466415405, Val Accuracy: 42.20000076293945
Iteration 600, Epoch 1, Loss: 1.8509196043014526, Accuracy: 38.03556442260742, Val Loss: 1.696663498878479, Val Accuracy: 41.400001525878906
Iteration 700, Epoch 1, Loss

Альтернативный менее гибкий способ обучения:

In [18]:
model = model_init_fn()
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=64, epochs=1, validation_data=(X_val, y_val))
model.evaluate(X_test, y_test)

766/766 ━━━━━━━━━━━━━━━━━━━━ 25s 31ms/step - loss: 1.9890 - sparse_categorical_accuracy: 0.3451 - val_loss: 1.7798 - val_sparse_categorical_accuracy: 0.3950
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 1.7638 - sparse_categorical_accuracy: 0.3985


[1.7682240009307861, 0.39730000495910645]

Перепишите реализацию трехслойной CNN с помощью tf.keras.Sequential API . Обучите модель двумя способами.

In [19]:
def model_init_fn():
    model = None
    ############################################################################
    # TODO: Construct a three-layer ConvNet using tf.keras.Sequential.         #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    model = tf.keras.Sequential([
        # Первый сверточный слой: 32 фильтра размером 5x5, активация ReLU
        tf.keras.layers.Conv2D(32, (5, 5), padding='same', activation='relu', input_shape=(32, 32, 3)),
        # Второй сверточный слой: 16 фильтров размером 3x3, активация ReLU
        tf.keras.layers.Conv2D(16, (3, 3), padding='same', activation='relu'),
        # Преобразование выходных данных сверточных слоев в плоский вектор
        tf.keras.layers.Flatten(),
        # Полносвязный слой с 10 нейронами и активацией softmax для классификации
        tf.keras.layers.Dense(10, activation='softmax')
    ])

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                            END OF YOUR CODE                              #
    ############################################################################
    return model

learning_rate = 5e-4
def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    # Создание оптимизатора Adam с указанной скоростью обучения
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return optimizer

# Обучение модели с помощью train_part34
train_part34(model_init_fn, optimizer_init_fn)

Z:\Programs\Python\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Iteration 0, Epoch 1, Loss: 2.303896427154541, Accuracy: 12.5, Val Loss: 2.468090772628784, Val Accuracy: 10.59999942779541
Iteration 100, Epoch 1, Loss: 1.870271921157837, Accuracy: 34.15841293334961, Val Loss: 1.6311159133911133, Val Accuracy: 44.20000076293945
Iteration 200, Epoch 1, Loss: 1.7217847108840942, Accuracy: 39.4511833190918, Val Loss: 1.4576530456542969, Val Accuracy: 51.20000076293945
Iteration 300, Epoch 1, Loss: 1.6273754835128784, Accuracy: 42.8311882019043, Val Loss: 1.3945655822753906, Val Accuracy: 52.39999771118164
Iteration 400, Epoch 1, Loss: 1.5556272268295288, Accuracy: 45.25794982910156, Val Loss: 1.343374490737915, Val Accuracy: 54.20000076293945
Iteration 500, Epoch 1, Loss: 1.504088282585144, Accuracy: 47.05588912963867, Val Loss: 1.2656952142715454, Val Accuracy: 56.0
Iteration 600, Epoch 1, Loss: 1.471628189086914, Accuracy: 48.34650421142578, Val Loss: 1.2444210052490234, Val Accuracy: 55.69999694824219
Iteration 700, Epoch 1, Loss: 1.4418513774871826,

In [20]:
model = model_init_fn()
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=64, epochs=1, validation_data=(X_val, y_val))
model.evaluate(X_test, y_test)

766/766 ━━━━━━━━━━━━━━━━━━━━ 13s 17ms/step - loss: 1.8044 - sparse_categorical_accuracy: 0.3556 - val_loss: 1.3728 - val_sparse_categorical_accuracy: 0.5140
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 1.3678 - sparse_categorical_accuracy: 0.5139


[1.3710508346557617, 0.5120999813079834]

# Использование Keras Functional API

Для реализации более сложных архитектур сети с несколькими входами/выходами, повторным использованием слоев, "остаточными" связями (residual connections) необходимо явно указать входные и выходные тензоры.

Ниже представлен пример для полносвязной сети.

In [21]:
def two_layer_fc_functional(input_shape, hidden_size, num_classes):
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    inputs = tf.keras.Input(shape=input_shape)
    flattened_inputs = tf.keras.layers.Flatten()(inputs)
    fc1_output = tf.keras.layers.Dense(hidden_size, activation='relu',
                                 kernel_initializer=initializer)(flattened_inputs)
    scores = tf.keras.layers.Dense(num_classes, activation='softmax',
                             kernel_initializer=initializer)(fc1_output)

    # Instantiate the model given inputs and outputs.
    model = tf.keras.Model(inputs=inputs, outputs=scores)
    return model

def test_two_layer_fc_functional():
    """ A small unit test to exercise the TwoLayerFC model above. """
    input_size, hidden_size, num_classes = 50, 42, 10
    input_shape = (50,)

    x = tf.zeros((64, input_size))
    model = two_layer_fc_functional(input_shape, hidden_size, num_classes)

    with tf.device(device):
        scores = model(x)
        print(scores.shape)

test_two_layer_fc_functional()

(64, 10)


In [22]:
input_shape = (32, 32, 3)
hidden_size, num_classes = 4000, 10
learning_rate = 1e-2

def model_init_fn():
    return two_layer_fc_functional(input_shape, hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.774383306503296, Accuracy: 14.0625, Val Loss: 2.9861907958984375, Val Accuracy: 12.300000190734863
Iteration 100, Epoch 1, Loss: 2.2660133838653564, Accuracy: 27.831066131591797, Val Loss: 1.9430962800979614, Val Accuracy: 37.5
Iteration 200, Epoch 1, Loss: 2.1008458137512207, Accuracy: 31.72418975830078, Val Loss: 1.8931944370269775, Val Accuracy: 38.60000228881836
Iteration 300, Epoch 1, Loss: 2.0197901725769043, Accuracy: 33.90261459350586, Val Loss: 1.877178430557251, Val Accuracy: 37.70000076293945
Iteration 400, Epoch 1, Loss: 1.9474018812179565, Accuracy: 35.89463806152344, Val Loss: 1.7702287435531616, Val Accuracy: 41.10000228881836
Iteration 500, Epoch 1, Loss: 1.8996343612670898, Accuracy: 37.06337356567383, Val Loss: 1.6529210805892944, Val Accuracy: 45.10000228881836
Iteration 600, Epoch 1, Loss: 1.867045283317566, Accuracy: 37.95756912231445, Val Loss: 1.691630482673645, Val Accuracy: 41.80000305175781
Iteration 700, Epoch 1, Loss: 1.84021008

Поэкспериментируйте с архитектурой сверточной сети. Для вашего набора данных вам необходимо получить как минимум 70% accuracy на валидационной выборке за 10 эпох обучения. Опишите все эксперименты и сделайте выводы (без выполнения данного пункта работы приниматься не будут).

Эспериментируйте с архитектурой, гиперпараметрами, функцией потерь, регуляризацией, методом оптимизации.  

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/BatchNormalization#methods https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dropout#methods

In [23]:
if tf.config.list_physical_devices('GPU'):
    print("GPU доступен!")
else:
    print("GPU не обнаружен. Код будет работать на CPU.")

GPU не обнаружен. Код будет работать на CPU.


In [24]:
class CustomConvNet(tf.keras.Model):
    def __init__(self):
        super(CustomConvNet, self).__init__()
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        # Сверточный слой с 64 фильтрами размером 3x3, padding 'same' и активацией ReLU
        self.conv1 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu')
        # Сверточный слой с 128 фильтрами размером 3x3, padding 'same' и активацией ReLU
        self.conv2 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu')
        # Слой максимального пулинга с размером окна 2x2
        self.max_pool = tf.keras.layers.MaxPooling2D((2, 2))
        # Слой Dropout с вероятностью 0.25
        self.dropout1 = tf.keras.layers.Dropout(0.25)
        # Сверточный слой с 256 фильтрами размером 3x3, padding 'same' и активацией ReLU
        self.conv3 = tf.keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu')
        # Сверточный слой с 512 фильтрами размером 3x3, padding 'same' и активацией ReLU
        self.conv4 = tf.keras.layers.Conv2D(512, (3, 3), padding='same', activation='relu')
        # Слой Dropout с вероятностью 0.25
        self.dropout2 = tf.keras.layers.Dropout(0.25)
        # Слой выравнивания (Flatten)
        self.flatten = tf.keras.layers.Flatten()
        # Полносвязный слой с 1024 нейронами и активацией ReLU
        self.fc1 = tf.keras.layers.Dense(1024, activation='relu')
        # Слой Dropout с вероятностью 0.5
        self.dropout3 = tf.keras.layers.Dropout(0.5)
        # Выходной полносвязный слой с 10 нейронами и активацией Softmax
        self.fc2 = tf.keras.layers.Dense(10, activation='softmax')

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################

    def call(self, input_tensor, training=False):
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        # Прохождение входного тензора через первый сверточный слой
        x = self.conv1(input_tensor)
        # Прохождение через второй сверточный слой
        x = self.conv2(x)
        # Применение максимального пулинга
        x = self.max_pool(x)
        # Применение Dropout после первого блока сверточных слоев
        x = self.dropout1(x, training=training)
        # Прохождение через третий сверточный слой
        x = self.conv3(x)
        # Прохождение через четвертый сверточный слой
        x = self.conv4(x)
        # Применение максимального пулинга
        x = self.max_pool(x)
        # Применение Dropout после второго блока сверточных слоев
        x = self.dropout2(x, training=training)
        # Выравнивание тензора в плоский вектор
        x = self.flatten(x)
        # Прохождение через первый полносвязный слой
        x = self.fc1(x)
        # Применение Dropout после первого полносвязного слоя
        x = self.dropout3(x, training=training)
        # Прохождение через выходной полносвязный слой
        x = self.fc2(x)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################
        return x


print_every = 500
num_epochs = 4

model = CustomConvNet()

def model_init_fn():
    return CustomConvNet()

def optimizer_init_fn():
    learning_rate = 1e-3
    return tf.keras.optimizers.Adam(learning_rate)

train_part34(model_init_fn, optimizer_init_fn, num_epochs=num_epochs, is_training=True)

Iteration 0, Epoch 1, Loss: 2.3331825733184814, Accuracy: 9.375, Val Loss: 8.704109191894531, Val Accuracy: 7.90000057220459
Iteration 500, Epoch 1, Loss: 1.6061919927597046, Accuracy: 42.08146286010742, Val Loss: 1.2006417512893677, Val Accuracy: 57.599998474121094
Iteration 1000, Epoch 2, Loss: 1.0469330549240112, Accuracy: 62.5, Val Loss: 0.9300834536552429, Val Accuracy: 68.19999694824219
Iteration 1500, Epoch 2, Loss: 0.9621782898902893, Accuracy: 65.93962097167969, Val Loss: 0.7653750777244568, Val Accuracy: 73.5
Iteration 2000, Epoch 3, Loss: 0.8046960234642029, Accuracy: 71.5918197631836, Val Loss: 0.7454738616943359, Val Accuracy: 74.19999694824219
Iteration 2500, Epoch 4, Loss: 0.7137347459793091, Accuracy: 75.00769805908203, Val Loss: 0.6826524138450623, Val Accuracy: 74.9000015258789
Iteration 3000, Epoch 4, Loss: 0.6867903470993042, Accuracy: 75.92461395263672, Val Loss: 0.6490383744239807, Val Accuracy: 77.60000610351562


Опишите все эксперименты, результаты. Сделайте выводы.

1. Структура сети включает в себя: сверточные сллои, максимальное объединение, droput, сглаживание и полностью связанные слои.
2. Были испытаны разнообразные оптимизаторы при одинаковых настройках: SGD, Adam, RMSprop. Наименее эффективным оказался SGD, показавший примерно 30% точности за 10 эпох. Adam и RMSprop показали схожие результаты, но в большинстве случаев Adam оказывался немного более точным.
3. Функции активации relu и sigmoid были протестированы на входном и скрытом слоях, в то время как на выходном слое были испытаны sigmoid и softmax. Наиболее низкий результат был достигнут при сочетании sigmoid + softmax + Adam - около 13 эпох необходимо было для достижения нужной точности. Наилучшая комбинация - relu + softmax + Adam - позволила достичь необходимого уровня точности всего за 4 эпохи. Это привело к сокращению числа эпох для обучения с 10 до 4.